In [1]:
%config IPCompleter.greedy=True

In [2]:
#Se importan las bibliotecas
import folium
from folium import Marker
import pandas as pd
import numpy as np

In [3]:
#Funcion para convertir de coordenadas (grados,minutos, segundos) a decimal
import re
def dms2dd(degrees, minutes, seconds, direction):
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'W':
        dd *= -1
    return dd;

def dd2dms(deg):
    d = int(deg)
    md = abs(deg - d) * 60
    m = int(md)
    sd = (md - m) * 60
    return [d, m, sd]

def parse_dms(dms):
    dms = dms.replace("\"","")
    dms = dms.replace("\'",",")
    dms = dms.replace("°",",")
    dms = dms.replace("´",",")
    parts = dms.split(",")
    
    #print(parts)
    lat = dms2dd(parts[0], parts[1], parts[2], parts[len(parts)-1])
    
    return (lat)

In [4]:
#Abrimos el archivo con la ubicacion de los EMAS
df = pd.read_excel("EMAS.xlsx")
df.head()

,ESTADO,NOMBRE,Latitud N,Longitud W,Altitud (m),Instalación en sitio
0,AGUASCALIENTES,CALVILLO,21°50'58'',102°42'44'',1618,2004-11-15
1,AGUASCALIENTES,PRESA 50 ANIV.,"22°11'21.5""","102°27'53.9""",2063.4,2012-12-05
2,AGUASCALIENTES,SIERRA FRIA,"22°16'11.2""","102°36'31.6""",2976.31,2012-12-08
3,BAJA CALIFORNIA,BAHÍA DE LOS ANGELES,"28°53'47""","113°33'37""",10,2000-04-06
4,BAJA CALIFORNIA,CATAVIÑA,29°43'38'',114°43'09'',514,2004-11-10


In [5]:
#Se agrega al final N a los valores de Latitud y W a los valores de Longitud
df["Latitud N"] = df["Latitud N"]+"\'N"
df["Longitud W"] = df["Longitud W"]+"\'W"
df.head()

,ESTADO,NOMBRE,Latitud N,Longitud W,Altitud (m),Instalación en sitio
0,AGUASCALIENTES,CALVILLO,21°50'58'''N,102°42'44'''W,1618,2004-11-15
1,AGUASCALIENTES,PRESA 50 ANIV.,"22°11'21.5""'N","102°27'53.9""'W",2063.4,2012-12-05
2,AGUASCALIENTES,SIERRA FRIA,"22°16'11.2""'N","102°36'31.6""'W",2976.31,2012-12-08
3,BAJA CALIFORNIA,BAHÍA DE LOS ANGELES,"28°53'47""'N","113°33'37""'W",10,2000-04-06
4,BAJA CALIFORNIA,CATAVIÑA,29°43'38'''N,114°43'09'''W,514,2004-11-10


In [6]:
#Se ejecuta la funcion para cada valor de las columnas de Latitud y longitud
df["Latitud N"] = df["Latitud N"].apply(parse_dms)
df["Longitud W"] = df["Longitud W"].apply(parse_dms)
df.head()

,ESTADO,NOMBRE,Latitud N,Longitud W,Altitud (m),Instalación en sitio
0,AGUASCALIENTES,CALVILLO,21.849444,-102.712222,1618,2004-11-15
1,AGUASCALIENTES,PRESA 50 ANIV.,22.189306,-102.464972,2063.4,2012-12-05
2,AGUASCALIENTES,SIERRA FRIA,22.269778,-102.608778,2976.31,2012-12-08
3,BAJA CALIFORNIA,BAHÍA DE LOS ANGELES,28.896389,-113.560278,10,2000-04-06
4,BAJA CALIFORNIA,CATAVIÑA,29.727222,-114.719167,514,2004-11-10


In [7]:
#Se crea una nueva columna con la combinacion de nombre con estado
df["Nombre Completo"] = df["NOMBRE"]+", "+df["ESTADO"]
df.head()

,ESTADO,NOMBRE,Latitud N,Longitud W,Altitud (m),Instalación en sitio,Nombre Completo
0,AGUASCALIENTES,CALVILLO,21.849444,-102.712222,1618,2004-11-15,"CALVILLO, AGUASCALIENTES"
1,AGUASCALIENTES,PRESA 50 ANIV.,22.189306,-102.464972,2063.4,2012-12-05,"PRESA 50 ANIV. , AGUASCALIENTES"
2,AGUASCALIENTES,SIERRA FRIA,22.269778,-102.608778,2976.31,2012-12-08,"SIERRA FRIA, AGUASCALIENTES"
3,BAJA CALIFORNIA,BAHÍA DE LOS ANGELES,28.896389,-113.560278,10,2000-04-06,"BAHÍA DE LOS ANGELES, BAJA CALIFORNIA"
4,BAJA CALIFORNIA,CATAVIÑA,29.727222,-114.719167,514,2004-11-10,"CATAVIÑA, BAJA CALIFORNIA"


In [9]:
cols = ["Nombre Completo", "Latitud N", "Longitud W", "Altitud (m)","Instalación en sitio"]
tabla = df[cols]
tabla.to_excel("Catalogo.xlsx")
tabla.head()

,Nombre Completo,Latitud N,Longitud W,Altitud (m),Instalación en sitio
0,"CALVILLO, AGUASCALIENTES",21.849444,-102.712222,1618,2004-11-15
1,"PRESA 50 ANIV. , AGUASCALIENTES",22.189306,-102.464972,2063.4,2012-12-05
2,"SIERRA FRIA, AGUASCALIENTES",22.269778,-102.608778,2976.31,2012-12-08
3,"BAHÍA DE LOS ANGELES, BAJA CALIFORNIA",28.896389,-113.560278,10,2000-04-06
4,"CATAVIÑA, BAJA CALIFORNIA",29.727222,-114.719167,514,2004-11-10


In [10]:
#Se crea un mapa cuyo que estara ubicado en las coordenadas de location, aprox el centro de Mexico
mapa =folium.Map(
    location = [25, -102],
    zoom_start=5,
    )
#Se agregan tipos de mapbox, junto con un controlador
folium.TileLayer('stamenterrain', name="Stamen Terrain").add_to(mapa)
folium.TileLayer('cartodbpositron', name="CartonDB Positron").add_to(mapa)
folium.LayerControl().add_to(mapa)

#Ciclo en donde se obtiene las coordenadas y los nombres de las estaciones, y se agrega como un marcador al mapa
for i in tabla["Nombre Completo"]:
    tooltip = i
    lat = float(tabla["Latitud N"][tabla["Nombre Completo"]==i])
    lon = float(tabla["Longitud W"][tabla["Nombre Completo"]==i])
    mapa.add_child(Marker(location=[lat,lon],popup=i,tooltip=tooltip))
mapa

In [11]:
#Se guarda el mapa con extension html
mapa.save("Mapa - Estaciones Meteorologicas.html")